**`This code shows how to use txtai to fine-tune the extractive QA model: 'ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA'.`**

This code is an adaption from: https://neuml.github.io/txtai/examples/ by txtai

Dataset: CGSQuAD

Dataset format: XLSX


* Dataset file name: CGSQuAD.xlsx
* Necessary columns: context, question, answers, answer_start
* Unecessary columns: pq_id


# Install dependencies



In [ ]:
%%capture
!pip install git+https://github.com/neuml/txtai datasets pandas
!pip install openpyxl
!pip install scikit-learn
!pip install torch
!pip install accelerate
!pip install transformers[torch]
!pip install numpy

#Preprocess CGSQuAD

In [ ]:
from datasets import load_dataset
from txtai.pipeline import HFTrainer

import pandas as pd
columns_to_read=["question", "context", "answers", "answer_start"]
df = pd.read_excel('/content/CGSQuAD.xlsx', usecols=columns_to_read)
trainer = HFTrainer()
# Convert the DataFrame to a list of dictionaries
data = df.to_dict(orient="records")

In [ ]:
df

,question,context,answers,answer_start
0,ما هي رسالة كلية الدراسات العليا؟,رسالة كلية الدراسات العليا هي العمل المخطط اله...,العمل المخطط الهادف الى المساهمة في تنمية إمكا...,30
1,ما هي غاية كلية الدراسات العليا؟,رسالة كلية الدراسات العليا هي العمل المخطط اله...,اتاحة فرص تعليم,159
2,ما هي أهداف كلية الدراسات العليا؟,رسالة كلية الدراسات العليا هي العمل المخطط اله...,اتاحة فرص تعليم,159
3,ما هي مهام كلية الدراسات العليا؟,رسالة كلية الدراسات العليا هي العمل المخطط اله...,الموافقة على برامج الدراسات العليا ووضع الأنظمة,278
4,ما هي واجبات كلية الدراسات العليا؟,رسالة كلية الدراسات العليا هي العمل المخطط اله...,الموافقة على برامج الدراسات العليا ووضع الأنظمة,278
...,...,...,...,...
1499,ما هو عنوان البريد؟,يكون موعد التقديم على كلية الدراسات العليا عاد...,اسم المنطقة التي يتواجد بها مكتب البريد,12310
1500,ما هو العنوان البريدي؟,يكون موعد التقديم على كلية الدراسات العليا عاد...,اسم المنطقة التي يتواجد بها مكتب البريد,12310
1501,ما هو الرمز البريدي؟,يكون موعد التقديم على كلية الدراسات العليا عاد...,رقم خاص يرمز الى بريد المنطقة,12368
1502,ما معنى الرمز البريدي؟,يكون موعد التقديم على كلية الدراسات العليا عاد...,رقم خاص يرمز الى بريد المنطقة,12368


In [ ]:
df=data

In [ ]:
data[0]

{'question': 'ما هي رسالة كلية الدراسات العليا؟',
 'context': 'رسالة كلية الدراسات العليا هي العمل المخطط الهادف الى المساهمة في تنمية إمكانات جامعة الكويت لتكون مؤسسة علمية تعليمية متميزة. أهداف كلية الدراسات العليا تشمل اتاحة فرص تعليم ومواكبة التطور العالمي وصنع المثقف العصري وتناول قضايا المجتمع الكويتي. مهام كلية الدراسات العليا تشمل الموافقة على برامج الدراسات العليا ووضع الأنظمة وتحديد أسس القبول وعمل التقويم الدوري وطرح برامج اختصاصية. كلية الدراسات العليا هي المؤسسة المسؤولة عن الدراسات العليا في جامعة الكويت تخطيطا وتنفيذا وتطويرا. العلاقة بين كلية الدراسات العليا والكليات الأخرى ومراكز البحث العلمي داخل الجامعة هي علاقة عضوية تخضع للتنسيق الأكاديمي. مجلس كلية الدراسات العليا هو الهيئة العليا لبحث شؤون الكلية ووضع سياساتها والمسؤول عن الاشراف على البرامج وتقويمها. يتكون مجلس كلية الدراسات العليا من عميد الكلية والعمداء المساعدون ورؤساء اللجان وأعضاء هيئة التدريس وأعضاء يسميهم مدير الجامعة. عميد كلية الدراسات العليا هو المسؤول أكاديميا واداريا عن تنفيذ ومتابعة خطط البرامج وله 

answers and answer_start should be wrapped together to match HFTrainer() format

In [ ]:
def modify_answer_start(data):
    new_data = []

    for row in data:
        modified_row = row.copy()
        answer_text = str(row['answers'])
        modified_row['answers'] = {#list wrapping
            'text': [answer_text],
            'answer_start': [row['answer_start']]
        }
        new_data.append(modified_row)

    return new_data
modified_data = modify_answer_start(df)

In [ ]:
print(data[0])
print(modified_data[0])

{'question': 'ما هي رسالة كلية الدراسات العليا؟', 'context': 'رسالة كلية الدراسات العليا هي العمل المخطط الهادف الى المساهمة في تنمية إمكانات جامعة الكويت لتكون مؤسسة علمية تعليمية متميزة. أهداف كلية الدراسات العليا تشمل اتاحة فرص تعليم ومواكبة التطور العالمي وصنع المثقف العصري وتناول قضايا المجتمع الكويتي. مهام كلية الدراسات العليا تشمل الموافقة على برامج الدراسات العليا ووضع الأنظمة وتحديد أسس القبول وعمل التقويم الدوري وطرح برامج اختصاصية. كلية الدراسات العليا هي المؤسسة المسؤولة عن الدراسات العليا في جامعة الكويت تخطيطا وتنفيذا وتطويرا. العلاقة بين كلية الدراسات العليا والكليات الأخرى ومراكز البحث العلمي داخل الجامعة هي علاقة عضوية تخضع للتنسيق الأكاديمي. مجلس كلية الدراسات العليا هو الهيئة العليا لبحث شؤون الكلية ووضع سياساتها والمسؤول عن الاشراف على البرامج وتقويمها. يتكون مجلس كلية الدراسات العليا من عميد الكلية والعمداء المساعدون ورؤساء اللجان وأعضاء هيئة التدريس وأعضاء يسميهم مدير الجامعة. عميد كلية الدراسات العليا هو المسؤول أكاديميا واداريا عن تنفيذ ومتابعة خطط البرامج وله أ

# Fine-tune the model on CGSQuAD



In [ ]:
from txtai.pipeline import HFTrainer
from sklearn.model_selection import train_test_split

#modified_data is a list of dictionaries
train_data, test_data = train_test_split(modified_data, test_size=0.2, random_state=42)
trainer = HFTrainer()
modelx, tokenizer = trainer("ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA", train_data, task="question-answering", num_train_epochs=10)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.319900
1000,0.838400
1500,0.770100
2000,0.726500
2500,0.719800
3000,0.694800
3500,0.678500
4000,0.682200
4500,0.672600


In [ ]:
save_directory = "/content/Model3"
modelx.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/Model3/tokenizer_config.json',
 '/content/Model3/special_tokens_map.json',
 '/content/Model3/vocab.txt',
 '/content/Model3/added_tokens.json',
 '/content/Model3/tokenizer.json')

# Test the model on CGSQuAD



In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

loaded_model = AutoModelForQuestionAnswering.from_pretrained(save_directory)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [ ]:
from transformers import pipeline

questions = pipeline("question-answering", model="ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA")
questions("من هو الطالب المقيد؟", " الطالب المقيد هو الطالب المستوفي لجميع الشروط اما الطالب الغير مقيد هو الطالب الغير مستوفي لشرط المعدل  ")

{'score': 0.8473714590072632,
 'start': 18,
 'end': 46,
 'answer': 'الطالب المستوفي لجميع الشروط'}

In [ ]:
from transformers import pipeline

questions = pipeline("question-answering", model="/content/Model3")
questions("من هو الطالب المقيد؟", " الطالب المقيد هو الطالب المستوفي لجميع الشروط اما الطالب الغير مقيد هو الطالب الغير مستوفي لشرط المعدل  ")

{'score': 0.9997518658638,
 'start': 18,
 'end': 46,
 'answer': 'الطالب المستوفي لجميع الشروط'}

In [ ]:
from transformers import pipeline

questions = pipeline("question-answering", model=modelx.to("cpu"), tokenizer=tokenizer)
questions(" من هو الطالب المقيد؟", " الطالب المقيد هو الطالب المستوفي لجميع الشروط اما الطالب الغير مقيد هو الطالب الغير مستوفي لشرط المعدل")

{'score': 0.9997518658638,
 'start': 18,
 'end': 46,
 'answer': 'الطالب المستوفي لجميع الشروط'}

#Evaluate the model on CGSQuAD

In [ ]:
def exact_match(predicted, ground_truth_list):
    return int(any([predicted.strip() == gt.strip() for gt in ground_truth_list]))

def f1_(predicted, ground_truth_list):
    pred_tokens = predicted.strip().split()
    f1_scores = []
    for ground_truth in ground_truth_list:
        gt_tokens = ground_truth.strip().split()
        common = set(pred_tokens) & set(gt_tokens)
        num_same = len(common)
        if num_same == 0:
            f1_scores.append(0)
        else:
            precision = 1.0 * num_same / len(pred_tokens)
            recall = 1.0 * num_same / len(gt_tokens)
            f1 = (2 * precision * recall) / (precision + recall)
            f1_scores.append(f1)
    return max(f1_scores)

em_scores = []
f1_scores = []

for row in test_data:
    context = row["context"]
    question = row["question"]
    ground_truth = row["answers"]["text"]

    prediction = questions(question, context)

    predicted_answer = prediction["answer"]
    em = exact_match(predicted_answer, ground_truth)
    f1 = f1_(predicted_answer, ground_truth)

    em_scores.append(em)
    f1_scores.append(f1)

In [ ]:
import numpy as np

print("Exact Match (EM) score:", np.mean(em_scores))
print("F1 score:", np.mean(f1_scores))

Exact Match (EM) score: 0.8770764119601329
F1 score: 0.9503181104278582


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
modelx.push_to_hub("AraELECTRA-CGSQuAD-QA-Model3")
tokenizer.push_to_hub("AraELECTRA-CGSQuAD-QA-Model3")

pytorch_model.bin:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/FatemahAlsubaiei/AraELECTRA-CGSQuAD-QA-Model3/commit/800b0a1d5709b3a45ab722fdd31a388ba4802655', commit_message='Upload tokenizer', commit_description='', oid='800b0a1d5709b3a45ab722fdd31a388ba4802655', pr_url=None, pr_revision=None, pr_num=None)